# dataset_schema_exploration.ipynb
# 📡 Satellite Communications Dataset – Schema Exploration

## Overview
This notebook explores the SatNOGS satellite communications dataset schema.
Goal: Understand the structure of all tables, their columns, and row counts,
so we can later prepare the dataset for machine learning tasks.

## Objectives:
- Inspect each table in the database (satnogs)
- Document column names and row counts
- Preview sample rows for context
- Identify relationships between tables
- Save schema summaries for reuse

In [1]:
# --- Install dependencies ---
%pip install sqlalchemy pandas pymysql tabulate

Note: you may need to restart the kernel to use updated packages.


In [2]:
# --- Imports ---
import sqlalchemy
import pandas as pd
from sqlalchemy import create_engine, inspect

print("SQLAlchemy version:", sqlalchemy.__version__)
print("Pandas version:", pd.__version__)

SQLAlchemy version: 2.0.44
Pandas version: 2.3.3


In [4]:
# --- Database connection ---
DB_USER = "root"
DB_PASSWORD = "123456789"
DB_HOST = "127.0.0.1"
DB_PORT = "3306"
DB_NAME = "satnogs"

engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
inspector = inspect(engine)

# --- List all tables ---
tables = inspector.get_table_names()
print(f"Tables in database ({len(tables)}):")
for i, table in enumerate(tables, 1):
    print(f"  {i:2d}. {table}")

Tables in database (15):
   1. base_antenna
   2. base_antennatype
   3. base_frequencyrange
   4. base_launch
   5. base_mode
   6. base_observation
   7. base_operator
   8. base_satellite
   9. base_satelliteentry
  10. base_satelliteidentifier
  11. base_station
  12. base_stationstatuslog
  13. base_stationtype
  14. base_telemetry
  15. base_transmitterentry


In [5]:
# --- Helper function to explore a table ---
def explore_table(table_name, preview_rows=5):
    """Return schema summary for a given table."""
    cols = [c['name'] for c in inspector.get_columns(table_name)]
    count = pd.read_sql(f"SELECT COUNT(*) AS total_rows FROM {table_name}", engine)['total_rows'][0]
    df_preview = pd.read_sql(f"SELECT * FROM {table_name} LIMIT {preview_rows}", engine)
    
    summary = {
        "table": table_name,
        "columns": cols,
        "row_count": count,
        "preview": df_preview
    }
    return summary

In [7]:
# --- Explore all relevant tables ---
tables_to_explore = [
    "base_antenna",
    "base_antennatype",
    "base_frequencyrange",
    "base_launch",
    "base_mode",
    "base_observation",
    "base_operator",
    "base_satellite",
    "base_satelliteentry",
    "base_satelliteidentifier",
    "base_station",
    "base_stationstatuslog",
    "base_stationtype",
    "base_telemetry",
    "base_transmitterentry"
]

schema_summaries = []
for tbl in tables_to_explore:
    summary = explore_table(tbl)
    schema_summaries.append(summary)
    print(f"\n=== {tbl} ===")
    print(f"Row count: {summary['row_count']:,}")
    print(f"Columns ({len(summary['columns'])}): {', '.join(summary['columns'][:5])}..." if len(summary['columns']) > 5 else f"Columns: {', '.join(summary['columns'])}")
    print("Preview:")
    display(summary["preview"].head())


=== base_antenna ===
Row count: 4,437
Columns: id, antenna_type_id, station_id
Preview:


,id,antenna_type_id,station_id
0,1,12,256
1,2,12,1536
2,7,10,771
3,8,13,1285
4,9,15,6



=== base_antennatype ===
Row count: 17
Columns: id, name
Preview:


,id,name
0,6,Cross Yagi
1,1,Dipole
2,3,Discone
3,12,Eggbeater
4,4,Ground Plane



=== base_frequencyrange ===
Row count: 5,311
Columns: id, min_frequency, max_frequency, antenna_id
Preview:


...

,id,name
0,1,RF



=== base_telemetry ===
Row count: 185
Columns: id, name, decoder, satellite_id
Preview:


,id,name,decoder,satellite_id
0,1,ISS AX.25,iss,28
1,2,STRAND-1 Telemetry,strand,87
2,3,UNISAT-6 Telemetry,us6,132
3,4,FOX-1A Telemetry,fox,178
4,5,QBEE Telemetry,qbee,227



=== base_transmitterentry ===
Row count: 9,869
Columns (29): id, uuid, description, uplink_low, uplink_high...
Preview:


,id,uuid,description,uplink_low,uplink_high,downlink_low,downlink_high,invert,baud,approved,...,uplink_mode_id,reviewed,reviewer_id,satellite_id,iaru_coordination,iaru_coordination_url,itu_notification,unconfirmed,receive_review_update,review_message
0,1,ZAKErADdWKpMiDjvKKhmmB,Mode U TLM,NaN,None,437125000,None,0,12.0,1,...,NaN,2019-04-18 05:39:53.343316,None,21,N/A,,"{""urls"": []}",0,0,None
1,3,ybJ86zjXzQxDReZ5skY56B,Mode H TLM,NaN,None,29352000,None,0,0.0,1,...,NaN,2019-04-18 05:39:53.343316,None,24,N/A,,"{""urls"": []}",0,0,None
2,5,Zqa2ebzyRRBffvwkLnjTVc,Mode U CW Beacon,NaN,None,435795000,None,0,0.0,1,...,NaN,2019-04-18 05:39:53.343316,None,25,N/A,,"{""urls"": []}",0,0,None
3,6,c4T33yxNiE8EAEc7V6LMQk,"Mode V/U APRS,BBS 9K6 FSK",145930000.0,None,435225000,None,1,9600.0,1,...,72.0,2019-04-18 05:39:53.343316,None,27,N/A,,"{""urls"": []}",0,0,None
4,7,maYGaaMWsSBeDDDMpcM9ES,Mode V/U BBS1 9K6 FSK,145850000.0,None,435225000,None,1,9600.0,1,...,72.0,2019-04-18 05:39:53.343316,None,27,N/A,,"{""urls"": []}",0,0,None


In [8]:
# --- Save schema summaries ---
import os

# Create reports directory if it doesn't exist
os.makedirs("../reports", exist_ok=True)

# Flatten into a DataFrame for CSV
schema_records = []
for s in schema_summaries:
    schema_records.append({
        "table": s["table"],
        "row_count": s["row_count"],
        "columns": ", ".join(s["columns"])
    })

schema_df = pd.DataFrame(schema_records)

# Save to CSV
csv_path = "../reports/schema_summary.csv"
schema_df.to_csv(csv_path, index=False)
print(f"✅ Schema summary saved to: {csv_path}")

# Save to Markdown
md_path = "../reports/schema_summary.md"
with open(md_path, "w", encoding="utf-8") as f:
    f.write("# 📊 SatNOGS Dataset Schema Summary\n\n")
    for s in schema_summaries:
        f.write(f"## Table: {s['table']}\n")
        f.write(f"- Row count: {s['row_count']:,}\n")
        f.write(f"- Columns: {', '.join(s['columns'])}\n\n")
        f.write("### Preview (first 5 rows)\n")
        f.write(s["preview"].head().to_markdown(index=False))
        f.write("\n\n---\n\n")
print(f"✅ Markdown report saved to: {md_path}")

✅ Schema summary saved to: ../reports/schema_summary.csv
✅ Markdown report saved to: ../reports/schema_summary.md


In [9]:
# --- Quick Analysis of Key Tables ---
print("📊 Key Table Analysis:")
print("=" * 50)

# Observation table stats
obs_stats = pd.read_sql("""
SELECT 
    MIN(start) AS first_observation,
    MAX(start) AS last_observation,
    COUNT(*) AS total_observations
FROM base_observation
""", engine)

print(f"📈 Observations: {obs_stats['total_observations'][0]:,} records")
print(f"📅 Date range: {obs_stats['first_observation'][0]} to {obs_stats['last_observation'][0]}")

# Station count
station_count = pd.read_sql("SELECT COUNT(*) FROM base_station", engine).iloc[0,0]
print(f"🏭 Stations: {station_count:,}")

# Satellite count
sat_count = pd.read_sql("SELECT COUNT(*) FROM base_satelliteentry", engine).iloc[0,0]
print(f"🛰️ Satellites: {sat_count:,}")

📊 Key Table Analysis:
📈 Observations: 12,546,241 records
📅 Date range: 2015-10-12 15:13:16 to 2025-11-12 09:59:41
🏭 Stations: 3,912
🛰️ Satellites: 9,759
